In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv


In [2]:
df = pd.read_csv('/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')

## Feature Engineering
### Changing categorical variables to 1's and 0's

In [27]:
df['Female'] = pd.Series(np.where(df.gender.values == 'Female', 1, 0),df.index)
df = df.drop('gender',axis=1)

### Converting yes and no.

In [ ]:
for i in ['Partner','Dependents','PhoneService','MultipleLines','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling','Churn']:
    df[i] = df[i].map({'Yes': 1, 'No': 0})
df = df.fillna(0)

### One-hot encoding with non-binary categorical columns

In [ ]:
df = df.join(pd.get_dummies(df['InternetService'],prefix='InternetService').drop('InternetService_No',axis=1))
df=df.drop('InternetService',axis=1)
df = df.join(pd.get_dummies(df['PaymentMethod'],prefix='PaymentMethod'))
df = df.join(pd.get_dummies(df['Contract'],prefix='Contract'))
df=df.drop(['PaymentMethod','Contract'],axis=1)

### Removing Multicollinearity

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
import seaborn as sns
for i in range(len(np.where(df.corr() > .6)[0])):
    if np.where(df.corr() > .6)[0][i] != np.where(df.corr() > .6)[1][i]:
        print(df.corr().columns[np.where(df.corr() > .6)[1][i]],df.corr().columns[np.where(df.corr() > .6)[0][i]])

In [ ]:
X = df.drop(['customerID','Churn'],axis=1)
vif = [[var,variance_inflation_factor(X.values, X.columns.get_loc(var))] for var in X.columns]
df[np.array(list(filter(lambda x: x[1] < 5, vif)))[:,0]]